In [40]:
from doekbase.data_api.sequence.assembly.api import AssemblyAPI
from doekbase.data_api.annotation.genome_annotation.api import GenomeAnnotationAPI
from doekbase.data_api.taxonomy.taxon.api import TaxonAPI
from doekbase.workspace.client import Workspace
from doekbase.data_api.core import ObjectAPI
from doekbase.handle.Client import AbstractHandle as handleClient

services = {"workspace_service_url": "https://ci.kbase.us/services/ws/",
            "shock_service_url": "https://ci.kbase.us/services/shock-api/",
            "handle_service_url": "https://ci.kbase.us/services/handle_service/"}

import os
import string

token = os.environ["KB_AUTH_TOKEN"]


In [41]:
genome_ref = 'ReferenceGenomeAnnotations/kb|g.166819'#6838/146'#ReferenceGenomeAnnotationsV5/kb|g.166819
working_directory ="./"

ga_api = GenomeAnnotationAPI(services, token=token, ref=genome_ref)

tax_api = ga_api.get_taxon()

asm_api = ga_api.get_assembly()

In [46]:
genome_name = str(ga_api.get_id())

print genome_name

valid_chars = "-_.(){0}{1}".format(string.ascii_letters, string.digits)
temp_file_name = ""
filename_chars = list()

for character in genome_name:
    if character in valid_chars:
        filename_chars.append(character)
    else:
        filename_chars.append("_")

if len(filename_chars) == 0:
    temp_file_name = "GenbankFile"
else:
    temp_file_name = "".join(filename_chars)+".gbk"

output_file = os.path.join(working_directory,temp_file_name)

contig_ids = asm_api.get_contig_ids()
contig_lengths = asm_api.get_contig_lengths(contig_ids)

628


In [60]:
def writeHeader(contig_id, contig_lengths, fulltax, tax_api, outFile):
    outFile.write("LOCUS       " + contig_id + "             " + str(contig_lengths[contig_id]) + " bp    " +"DNA\n")
    sn = tax_api.get_scientific_name()
    outFile.write("DEFINITION  " + sn + " genome.\n")
    outFile.write("SOURCE      " + sn + "\n")
    outFile.write("  ORGANISM  " + sn + "\n")

    fulltax = tax_api.get_scientific_lineage();
    fulltaxstring = ';'.join(fulltax)
    print fulltax

    if (fulltax):
        formatTax = ""

        counter = 0
        index = 0
        while (index < len(fulltax)):
            formatTax.join(fulltax[index])
            if (index < len(fulltax) - 1):
                formatTax.join(" ")
            counter=counter+ len(fulltax[index]) + 1
            index=index+1

            if (counter >= 65 or len(fulltaxstring) < 80):
                formatTax.join("\n")
                formatTax.join("            ")
                counter = 0

        outFile.write("            " + formatTax + ".\n")

    outFile.write("COMMENT     Exported from the DOE KnowledgeBase.\n")

    outFile.write("FEATURES             Location/Qualifiers\n")
    outFile.write("     source          1.." + str(contig_lengths[contig_id]) + "\n")
    outFile.write("                     /organism=\"" + tax_api.get_scientific_name() + "\"\n")
    outFile.write("                     /mol_type=\"DNA\"\n") 

In [77]:
def writeFeatures(ga_api, regions, outFile):
    feature_ids = ga_api.get_feature_ids(filters={"region_list":regions})
    features = ga_api.get_features(feature_ids['by_type']['gene'])
    print len(features)
    
    for feat in features:
        
        print feat
        print features[feat]
        
        if (feat['type'] == "CDS") :
            outFile.write("     gene            ")
        else :
            if (function.indexOf("tRNA") != -1) :
                outFile.write("     tRNA            ")
            else :
                outFile.write("     misc_RNA        ")

        #out = writeCDS(location, outFile)

        if (feat['type'] == "CDS") :
            outFile.write("     CDS             ")
            #writeCDS(location, outFile)
            outFile.write("                     /gene=\"" + id + "\"\n")

        #outFile.write("                     /note=\"" + formatNote)
        #out += "                     /codon_start=1\n"
        #out += "                     /transl_table=11\n"
        #outFile.write("                     /product=\"" + id + "\"\n")
        outFile.write("                     /function=\"" + formatFunction)

        if (feat['type'] == "CDS") :
            str = "                     /protein_id=\"" + id + "\"\n"
            outFile.write(str)

        aliases = cur.getAliases()
        if (aliases != null) :
            for s in aliases :
                outFile.write("                     /db_xref=\"id:" + s + "\"\n")

        proteinTranslation = cur['amino_acid_sequence']
        if (proteinTranslation != null):
            outFile.write("                     /translation=\"" + formatString(proteinTranslation, 44, 58));

In [74]:
def writeCDS():
    added = 0
    complement = false
    join = false
    for n in range(0, len(location)) :
        now4 = location[n]
        if (added == 0 and now4['strand'] == "-") :
            outFile.write("complement(")
            complement = true

        if (len(location) > 1) :
            if (added == 0):
                outFile.write("join(")
            join = true
        
        #outpu location info
        #if (complement is false):
        #    outFile.write(now4.getE2() + ".." + (now4.getE2() + (long) now4.getE4() - 1))
        #else :
        #    outFile.write((now4.getE2() - (long) now4.getE4() + 1) + ".." + now4.getE2())

        #if (location.size() > 0 and n < location.size() - 1)
        #    outFile.write(",")
        added = added + 1

    if (complement and join):
        outFile.write("))\n")
    elif (complement or join):
        outFile.write(")\n")
    else:
        outFile.write("\n")

In [75]:
def formatString(s, one, two) :
    s = s.replace("\"", "")
    out = ""
    first = true
    for start in range(0, s.length()) :
        if (first) :
            lista = [len(s), start+one]
            last = min(lista)
            isLast = false
            if (last == len(s)):
                isLast = true
            out.append(s[start : last])
            if (isLast):
                out.append("\"\n")
            else :
                out.append("\n")
            
            first = false;
            start = start+one;
        else :
            lista = [len(s), start+two]
            last = min(lista)
            out.append("                     ")
            isLast = false
            if (last == len(s)):
                isLast = true
            out.append(s[start: last])
            start = start + two
            if (isLast) :
                out.append("\"\n")            
            else:
                out.append("\n");
    out

In [78]:

with open(output_file, "w") as outFile:

    #
    #create per-contig section in gbk file
    #
    for contig_id in contig_ids:

        start=1
        stop=contig_lengths[contig_id]

        print contig_id
        writeHeader(contig_id, contig_lengths, fulltax, tax_api, outFile)
        regions = []
        contig_add = {"contig_id": c, "start": start, "length": stop-start, "strand": "+"}
        contig_add2 = {"contig_id": c, "start": stop, "length": stop-start, "strand": "-"}        
        regions.append(contig_add)
        regions.append(contig_add2)

        writeFeatures(ga_api, regions, outFile)

        #write contig sequence

        break

outFile.close()    
#raise IOError("The Assembly associated with this GenomeAnnotation does not have a fasta_ref to shock or sequences in the contigs. A Genbank file can not be created.  Likely dueto the ContigSet being too large.")



kb|g.166819.c.7
[u'cellular organisms', u'Eukaryota', u'Viridiplantae', u'Chlorophyta', u'prasinophytes', u'Mamiellophyceae', u'Mamiellales', u'Bathycoccaceae', u'Ostreococcus']
